In [1]:
import os
import json
import pandas as pd
import numpy as np
heroes_data_file="purchase_data.json"
heroes_df= pd.read_json(heroes_data_file)


__Player Count__

In [2]:
total_unique_players = heroes_df['SN'].nunique()
#total_unique_players
players_df = pd.DataFrame([{'Total Players': total_unique_players}])
players_df.set_index('Total Players', inplace = True)
players_df


""
Total Players
573


__Purchase Totals__

In [3]:
# Number of Unique Items

total_unique_items = heroes_df['Item Name'].nunique()

# total puchases

total_purchases = heroes_df['Price'].count()

#total Revenue

total_revenue = round(heroes_df['Price'].sum(),2)

#avg price

avg_price = round(total_revenue /total_purchases, 2)

#create dataframe for values

pur_analysis = pd.DataFrame([{
    
    "Number of Unique Items": total_unique_items,
    'Average Purchase Price': avg_price,
    'Total Purchases': total_purchases,
    'Total Revenue': total_revenue
}])

pur_analysis.style.format({'Average Purchase Price': '${:.2f}', 'Total Revenue': '${:,.2f}'})

,Average Purchase Price,Number of Unique Items,Total Purchases,Total Revenue
0,$2.93,179,780,"$2,286.33"


__Player Analysis by Gender__<br>
Top level purchase analysis. No suprises that > 80 % of players in this genre are male

In [4]:
# Gender Data
# unique sn values by gender
uniq_players_gen_df=pd.DataFrame(heroes_df.groupby('Gender')['SN'].nunique())
uniq_players_gen_df.rename(columns = {'index': 'Gender', 'SN': '# of Players'}, inplace = True)

uniq_players_gen_df['% of Players'] = uniq_players_gen_df['# of Players']/total_unique_players * 100

uniq_players_gen_df
uniq_players_gen_df.style.format({"% of Players": "{:.1f}%"})

,# of Players,% of Players
Gender,,
Female,100,17.5%
Male,465,81.2%
Other / Non-Disclosed,8,1.4%


__Purchase Analysis by Gender__<br>
Purchases per player are almost identical between amongst all genders

In [5]:
#gender analysis
#using unique player counts from above
#counts of transactions
count_by_gen = pd.DataFrame(heroes_df.groupby('Gender')['Gender'].count())
count_by_gen.rename(columns = {'Gender': '# of Purchases'}, inplace=True)

#total sales
total_by_gen = pd.DataFrame(heroes_df.groupby('Gender')['Price'].sum())
total_by_gen.rename(columns = {'Price': 'Total Purchase Value'}, inplace=True)
# create sub frame to merge to
gender_analysis = pd.merge(uniq_players_gen_df[["# of Players"]],count_by_gen, left_index = True, right_index = True)
gender_analysis['Purchases per Player'] = gender_analysis_fnl['# of Purchases']/gender_analysis_fnl['# of Players']

#merge total sales
gender_analysis_fnl = pd.merge(gender_analysis,total_by_gen, left_index = True, right_index = True)
# add average price
gender_analysis_fnl['Average Purchase Price'] = gender_analysis_fnl['Total Purchase Value']/gender_analysis_fnl['# of Purchases']
#add normalized totals prices
gender_analysis_fnl['Normalized Totals'] = gender_analysis_fnl['Total Purchase Value']/gender_analysis_fnl['# of Players']


gender_analysis_fnl
#format
gender_analysis_fnl.style.format({'Total Purchase Value': '${:.2f}', 'Average Purchase Price': '${:.2f}', 'Normalized Totals': '${:.2f}'})





NameError: name 'gender_analysis_fnl' is not defined

__Purchase Analysis by Age__<br>
10-14 and 35-39 are the age groups most likely to make repeat purchases


In [14]:
#sales by age

#binning age data 
heroes_df.loc[(heroes_df['Age'] < 10), 'age_bin'] = "< 10"
heroes_df.loc[(heroes_df['Age'] >= 10) & (heroes_df['Age'] <= 14), 'age_bin'] = "10 - 14"
heroes_df.loc[(heroes_df['Age'] >= 15) & (heroes_df['Age'] <= 19), 'age_bin'] = "15 - 19"
heroes_df.loc[(heroes_df['Age'] >= 20) & (heroes_df['Age'] <= 24), 'age_bin'] = "20 - 24"
heroes_df.loc[(heroes_df['Age'] >= 25) & (heroes_df['Age'] <= 29), 'age_bin'] = "25 - 29"
heroes_df.loc[(heroes_df['Age'] >= 30) & (heroes_df['Age'] <= 34), 'age_bin'] = "30 - 34"
heroes_df.loc[(heroes_df['Age'] >= 35) & (heroes_df['Age'] <= 39), 'age_bin'] = "35 - 39"
heroes_df.loc[(heroes_df['Age'] >= 40), 'age_bin'] = "> 40"

# count users in bin
count_age = pd.DataFrame(heroes_df.groupby('age_bin')['SN'].count())
count_age.rename(columns = {'SN': '# of Purchases'}, inplace=True)

# average purchase 
avg_price_age = pd.DataFrame(heroes_df.groupby('age_bin')['Price'].mean())
avg_price_age.rename(columns = {'Price': 'Average Purchase Price'}, inplace=True)

#total puchases
tot_pur_age = pd.DataFrame(heroes_df.groupby('age_bin')['Price'].sum())
tot_pur_age.rename(columns = {'Price': 'Total Purchase Value'}, inplace=True)


# unique players count  and merge

unique_age_users = pd.DataFrame(heroes_df.groupby('age_bin')['SN'].nunique())
unique_age_users.rename(columns = {'SN': '# of Players'}, inplace=True)


#build master data frame - one merge at the time so I can keep up with what I'm doing
master_age_df_1 = pd.merge(count_age, unique_age_users, left_index = True, right_index = True)

master_age_df_2= pd.merge(master_age_df_1,avg_price_age, left_index = True, right_index = True)
master_age_df_final= pd.merge(master_age_df_2,tot_pur_age, left_index = True, right_index = True)


master_age_df_final['Normalized Totals'] = master_age_df_final['Total Purchase Value']/master_age_df_final['# of Players']
master_age_df_final['Purchases per Player'] = master_age_df_final['# of Purchases']/master_age_df_final['# of Players']
#format final
master_age_df_final
master_age_df_final.style.format({'Average Purchase Price': '${:.2f}', 'Total Purchase Value': '${:.2f}', 'Normalized Totals': '${:.2f}'})


,# of Purchases,# of Players,Average Purchase Price,Total Purchase Value,Normalized Totals,Purchases per Player
age_bin,,,,,,
10 - 14,35,23,$2.77,$96.95,$4.22,1.52174
15 - 19,133,100,$2.91,$386.42,$3.86,1.33
20 - 24,336,259,$2.91,$978.77,$3.78,1.2973
25 - 29,125,87,$2.96,$370.33,$4.26,1.43678
30 - 34,64,47,$3.08,$197.25,$4.20,1.3617
35 - 39,42,27,$2.84,$119.40,$4.42,1.55556
< 10,28,19,$2.98,$83.46,$4.39,1.47368
> 40,17,11,$3.16,$53.75,$4.89,1.54545


__Top Spenders__

In [8]:
# Top Spenders
#calculate variable dfs
total_amount_by_player = pd.DataFrame(heroes_df.groupby('SN')['Price'].sum())
total_amount_by_player.rename(columns = {'Price': 'Total Purchase Value'}, inplace=True)


purchases_by_player = pd.DataFrame(heroes_df.groupby('SN')['Price'].count())
purchases_by_player.rename(columns = {'Price': 'Total Purchases'}, inplace=True)

avg_purchase_by_player = pd.DataFrame(heroes_df.groupby('SN')['Price'].mean())
avg_purchase_by_player.rename(columns = {'Price': 'Average Purchase Price'}, inplace=True)

#append variable dfs with renamed columns
top_spend_df_1 = pd.merge(total_amount_by_player, purchases_by_player, left_index = True, right_index = True)
top_spend_df_final = pd.merge(top_spend_df_1,avg_purchase_by_player, left_index=True, right_index=True)

#sort on total spend
top_spend_df_final.sort_values('Total Purchase Value', ascending = False, inplace=True)
#top 5 values
top_spend_df_final = top_spend_df_final.head(5)

top_spend_df_final.style.format({'Total Purchase Value': '${:.2f}', 'Average Purchase Price': '${:.2f}'})

,Total Purchase Value,Total Purchases,Average Purchase Price
SN,,,
Undirrala66,$17.06,5,$3.41
Saedue76,$13.56,4,$3.39
Mindimnya67,$12.74,4,$3.18
Haellysu29,$12.73,3,$4.24
Eoda93,$11.58,3,$3.86


__Top Selling Items__

In [7]:
# Top selling Items
#calculate variable dfs
total_amount_by_item = pd.DataFrame(heroes_df.groupby('Item Name')['Price'].sum())
total_amount_by_item.rename(columns = {'Price': 'Total Revenue'}, inplace=True)


purchases_by_item = pd.DataFrame(heroes_df.groupby('Item Name')['Price'].count())
purchases_by_item.rename(columns = {'Price': 'Total Purchases'}, inplace=True)

price_by_item = pd.DataFrame(heroes_df.groupby('Item Name')['Price'].mean())
price_by_item.rename(columns = {'Price': 'Purchase Price'}, inplace=True)

#append variable dfs with renamed columns
top_items_df_1 = pd.merge(total_amount_by_item, purchases_by_item, left_index = True, right_index = True)
top_items_df_final = pd.merge(top_items_df_1,price_by_item, left_index=True, right_index=True)

#sort on total spend
top_items_df_final.sort_values('Total Purchases', ascending = False, inplace=True)
#top 5 values
top_items_df_final = top_items_df_final.head(5)

top_items_df_final.style.format({'Total Revenue': '${:.2f}', 'Purchase Price': '${:.2f}'})


,Total Revenue,Total Purchases,Purchase Price
Item Name,,,
Final Critic,$38.60,14,$2.76
Arcane Gem,$24.53,11,$2.23
"Betrayal, Whisper of Grieving Widows",$25.85,11,$2.35
Stormcaller,$34.65,10,$3.46
Woeful Adamantite Claymore,$11.16,9,$1.24


__ Most Profitable Items__

In [6]:
#most profitable items
#calculate variable dfs
total_amount_by_item = pd.DataFrame(heroes_df.groupby('Item Name')['Price'].sum())
total_amount_by_item.rename(columns = {'Price': 'Total Revenue'}, inplace=True)


purchases_by_item = pd.DataFrame(heroes_df.groupby('Item Name')['Price'].count())
purchases_by_item.rename(columns = {'Price': 'Total Purchases'}, inplace=True)

price_by_item = pd.DataFrame(heroes_df.groupby('Item Name')['Price'].mean())
price_by_item.rename(columns = {'Price': 'Purchase Price'}, inplace=True)

#append variable dfs with renamed columns
top_items_df_1 = pd.merge(total_amount_by_item, purchases_by_item, left_index = True, right_index = True)
top_items_df_final = pd.merge(top_items_df_1,price_by_item, left_index=True, right_index=True)

#sort on total spend
top_items_df_final.sort_values('Total Revenue', ascending = False, inplace=True)
#top 5 values
top_items_df_final = top_items_df_final.head(5)

top_items_df_final.style.format({'Total Revenue': '${:.2f}', 'Purchase Price': '${:.2f}'})

,Total Revenue,Total Purchases,Purchase Price
Item Name,,,
Final Critic,$38.60,14,$2.76
Retribution Axe,$37.26,9,$4.14
Stormcaller,$34.65,10,$3.46
Spectral Diamond Doomblade,$29.75,7,$4.25
Orenmir,$29.70,6,$4.95


In [7]:
count_by_gen = pd.DataFrame(heroes_df.groupby('Gender')['Gender'].count())
count_by_gen.rename(columns = {'Gender': '# of Purchases'}, inplace=True)
count_by_gen

,# of Purchases
Gender,
Female,136
Male,633
Other / Non-Disclosed,11
